# Green and Red Candle Project
## Coded by Victor Zheng & Supervised under Mohammed Hasan

In [1]:
"""
import time
tic = time.perf_counter()
npData = df.values
toc = time.perf_counter()
print(f"time to use .values is: {toc-tic:0.4f} seconds") #0.1871 seconds
"""
"""
#we learned that using df.iterrows to convert to lise using append is a lot slower than using df.values
npList = []
tic = time.perf_counter()
for index,row in df.iterrows():
    npList.append(row)
toc = time.perf_counter()
print(f"time to use append is: {toc-tic:0.4f} seconds") #18.4474 seconds
"""
"""
npList = [None] * len(df)
tic = time.perf_counter()
for index,row in df.iterrows():
    npList.append(row)
toc = time.perf_counter()
print(f"time to use initialized with setting is: {toc-tic:0.4f} seconds") #18.3087 seconds
"""

'\nnpList = [None] * len(df)\ntic = time.perf_counter()\nfor index,row in df.iterrows():\n    npList.append(row)\ntoc = time.perf_counter()\nprint(f"time to use initialized with setting is: {toc-tic:0.4f} seconds") #18.3087 seconds\n'

## Get minimum value for call/put between 9:46AM and the timecallmax for that day

In [2]:
#function that compares if time 1 is < time 2
def beforeTime(time1,time2):
    
    if len(time1) != 11 or len(time1) != 12:
        time1 = "0" + time1
    if len(time2) != 11 or len(time2) != 12:
        time2 = "0" + time2
    
    #check last 2 chars to see if AM or PM
    if time1[-2:] == "AM":
        t1IsAM = True
    else:
        t1IsAM = False
    if time2[-2:] == "AM":
        t2IsAM = True
    else:
        t2IsAM = False
    
    if t1IsAM and not t2IsAM:
        return True
    elif t1IsAM and t2IsAM:
        return True if time1 < time2 else False
    elif not t1IsAM and t2IsAM:
        return False
    else: #t1 is pm and t2 is pm
        return True if time1 < time2 else False

#sample test cases - expected TFFF
print(beforeTime("10:05:00 AM","3:59:00 PM"), beforeTime("10:05:00 PM","3:59:00 PM"), beforeTime("10:05:00 AM","3:59:00 AM"),beforeTime("10:05:00 PM","3:59:00 AM"))

def afterTime(time1,time2):
    if time1 == time2:
        return False
    return not beforeTime(time1,time2)

True True True False


In [3]:
import pandas as pd
import numpy as np
import math
#df = pd.read_csv('Jan 2020 - Dec 2020 merged SPY - Copy.csv')
#df = pd.read_csv('Jan 2020 - Dec 2020 merged SPY.csv')
df = pd.read_csv('2020-mergedData.csv')
#df = pd.read_csv('Even-smaller-dataframe.csv')

#npData = df.to_numpy()
npData = df.to_numpy(dtype="O")
        
N = len(df)

#40% training
#30% validation
#30% cross-validation
#trainingSlice = int(0.4*N)
#validationSlice = int(0.7*N)
#crossValidationSlice = N

#50% training
#50% validation
trainingSlice = int(0.5*N)
validationSlice = N

trainingSet = npData[0:trainingSlice]
validationSet = npData[trainingSlice:validationSlice]
#crossValidationSet = npData[validationSlice:crossValidationSlice]

In [4]:
df

,Date,TimeBarStart,Ticker,CallPut,Strike,ExpirationDate,OpenTradePrice,HighTradePrice,LowTradePrice,CloseTradePrice,UnderOpenBidPrice,UnderOpenAskPrice,UnderCloseBidPrice,UnderCloseAskPrice,VWAP,Volume,TotalTrades
0,20200103,9:30:00 AM,SPY,P,320.5,20200103,0.57,0.60,0.47,0.47,321.13,321.15,321.55,321.56,0.5865,241,17
1,20200103,9:30:00 AM,SPY,P,305.0,20200103,0.01,0.01,0.01,0.01,321.13,321.16,321.13,321.16,0.0100,6,1
2,20200103,9:30:00 AM,SPY,P,317.0,20200103,0.12,0.12,0.10,0.10,321.18,321.20,321.33,321.34,0.1018,11,2
3,20200103,9:30:00 AM,SPY,P,313.0,20200103,0.02,0.02,0.02,0.02,321.44,321.45,321.44,321.45,0.0200,1,1
4,20200103,9:30:00 AM,SPY,P,321.5,20200103,0.98,0.98,0.82,0.82,321.14,321.16,321.47,321.48,0.9241,279,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885432,20201231,4:14:00 PM,SPY,C,374.0,20201231,0.20,0.25,0.18,0.25,374.01,374.04,374.09,374.10,0.2030,796,38
1885433,20201231,4:14:00 PM,SPY,C,371.0,20201231,3.12,3.12,3.12,3.12,374.09,374.10,374.09,374.10,3.1200,2,1
1885434,20201231,4:14:00 PM,SPY,C,372.0,20201231,2.07,2.10,2.03,2.10,374.02,374.04,374.06,374.09,2.0618,17,5
1885435,20201231,4:14:00 PM,SPY,C,375.0,20201231,0.01,0.01,0.01,0.01,374.07,374.10,374.07,374.10,0.0100,10,1


In [5]:
import os
os.getcwd()
#here1

'C:\\Users\\Victor\\Desktop\\0-Leetcode\\Desktop\\Projects\\CandleAnalysis'

### Use the dataset with powershell data to calculate the candle type (closetradeprice - opentradeprice @9:30)

In [6]:
import os
directory = r"C:\Users\Victor\Desktop\0-Leetcode\Desktop\Projects\CandleAnalysis\Spy-Powershell"

#dateToCandle = {} #maps date to candleType
dateToCandle = {} #maps date to (date,time,candleType,StockPrice)

for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        #print(os.path.join(directory, filename))
        df1 = pd.read_csv(os.path.join(directory, filename))
        numDf1 = df1.to_numpy()
        for row in numDf1:
            #print(row)
            dateTime = row[1].split(" ")
            time = dateTime[1]
            if time == "9:30":
                candleType = "GREEN" if row[5] - row[2] >= 0 else "RED"
                #convert date to yyyymmdd
                date = dateTime[0]
                date = date.split("/")
                year = date[2]
                month = date[0]
                day = date[1]
                if len(month) == 1:
                    month = "0" + month
                if len(day) == 1:
                    day = "0" + day
                date = int(year + month + day)
                #time = "9:36:00 AM" #change the time to 9:36AM to later grab the OpenTradePrice (OTP) at that time
                dateToCandle[date] = (date,time,candleType,row[5])

In [7]:
for date in dateToCandle:
    print("date={},candle={}".format(date,dateToCandle[date]))

date=20160218,candle=(20160218, '9:30', 'RED', 192.76)
date=20160219,candle=(20160219, '9:30', 'GREEN', 191.35)
date=20160222,candle=(20160222, '9:30', 'GREEN', 193.94)
date=20160223,candle=(20160223, '9:30', 'RED', 193.96)
date=20160224,candle=(20160224, '9:30', 'RED', 190.14)
date=20160225,candle=(20160225, '9:30', 'GREEN', 194.16)
date=20160226,candle=(20160226, '9:30', 'RED', 196.46)
date=20160229,candle=(20160229, '9:30', 'RED', 195.07)
date=20160301,candle=(20160301, '9:30', 'RED', 194.98)
date=20160302,candle=(20160302, '9:30', 'RED', 197.7)
date=20160303,candle=(20160303, '9:30', 'RED', 198.66)
date=20160304,candle=(20160304, '9:30', 'RED', 199.69)
date=20160307,candle=(20160307, '9:30', 'GREEN', 199.3911)
date=20160308,candle=(20160308, '9:30', 'RED', 199.17)
date=20160309,candle=(20160309, '9:30', 'RED', 199.32)
date=20160310,candle=(20160310, '9:30', 'GREEN', 200.18)
date=20160311,candle=(20160311, '9:30', 'GREEN', 201.4599)
date=20160314,candle=(20160314, '9:30', 'GREEN', 2

In [8]:
#len(trainingSet[0])
df

,Date,TimeBarStart,Ticker,CallPut,Strike,ExpirationDate,OpenTradePrice,HighTradePrice,LowTradePrice,CloseTradePrice,UnderOpenBidPrice,UnderOpenAskPrice,UnderCloseBidPrice,UnderCloseAskPrice,VWAP,Volume,TotalTrades
0,20200103,9:30:00 AM,SPY,P,320.5,20200103,0.57,0.60,0.47,0.47,321.13,321.15,321.55,321.56,0.5865,241,17
1,20200103,9:30:00 AM,SPY,P,305.0,20200103,0.01,0.01,0.01,0.01,321.13,321.16,321.13,321.16,0.0100,6,1
2,20200103,9:30:00 AM,SPY,P,317.0,20200103,0.12,0.12,0.10,0.10,321.18,321.20,321.33,321.34,0.1018,11,2
3,20200103,9:30:00 AM,SPY,P,313.0,20200103,0.02,0.02,0.02,0.02,321.44,321.45,321.44,321.45,0.0200,1,1
4,20200103,9:30:00 AM,SPY,P,321.5,20200103,0.98,0.98,0.82,0.82,321.14,321.16,321.47,321.48,0.9241,279,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885432,20201231,4:14:00 PM,SPY,C,374.0,20201231,0.20,0.25,0.18,0.25,374.01,374.04,374.09,374.10,0.2030,796,38
1885433,20201231,4:14:00 PM,SPY,C,371.0,20201231,3.12,3.12,3.12,3.12,374.09,374.10,374.09,374.10,3.1200,2,1
1885434,20201231,4:14:00 PM,SPY,C,372.0,20201231,2.07,2.10,2.03,2.10,374.02,374.04,374.06,374.09,2.0618,17,5
1885435,20201231,4:14:00 PM,SPY,C,375.0,20201231,0.01,0.01,0.01,0.01,374.07,374.10,374.07,374.10,0.0100,10,1


## Trim dataset

In [9]:
#Delete rows from trainingSet that don't have the same StrikePrice to speed up our iteration of the df
trainingSetTrimmed = [] #np.zeros([17,]) #np.empty([17])
prevDate = trainingSet[0][0]
for date,time,ticker,callput,strike,expiration,OTP,HTP,LTP,CTP,UOBP,UOAP,UCBP,UCAP,VWAP,Vol,TT in trainingSet:
    candleType = dateToCandle[date][2]
    StockPrice = dateToCandle[date][3]
    strikePrice = math.ceil(StockPrice) if candleType == "GREEN" else math.floor(StockPrice)
    #print("strike={},strikePrice={}".format(strike,strikePrice))
    if strike == strikePrice: #dateToCandle maps date to (date,time,candleType,StockPrice)
        #print("appending: {}".format(trainingSetTrimmed))
        toAppend = np.array([date,time,ticker,callput,strike,expiration,OTP,HTP,LTP,CTP,UOBP,UOAP,UCBP,UCAP,VWAP,Vol,TT],dtype="O") #note we're using Object datatype so there are ints and str
        trainingSetTrimmed.append(toAppend)
trainingSet = np.array(trainingSetTrimmed)

In [10]:
#Delete rows from validationSet that don't have the same StrikePrice to speed up our iteration of the df
validationSetTrimmed = [] #np.zeros([17,]) #np.empty([17])
prevDate = validationSet[0][0]
for date,time,ticker,callput,strike,expiration,OTP,HTP,LTP,CTP,UOBP,UOAP,UCBP,UCAP,VWAP,Vol,TT in validationSet:
    candleType = dateToCandle[date][2]
    StockPrice = dateToCandle[date][3]
    strikePrice = math.ceil(StockPrice) if candleType == "GREEN" else math.floor(StockPrice)
    #print("strike={},strikePrice={}".format(strike,strikePrice))
    if strike == strikePrice: #dateToCandle maps date to (date,time,candleType,StockPrice)
        #print("appending: {}".format(validationSetTrimmed))
        toAppend = np.array([date,time,ticker,callput,strike,expiration,OTP,HTP,LTP,CTP,UOBP,UOAP,UCBP,UCAP,VWAP,Vol,TT],dtype="O") #note we're using Object datatype so there are ints and str
        validationSetTrimmed.append(toAppend)
validationSet = np.array(validationSetTrimmed)

In [11]:
#Remove 4:00PM rows from the dataset
dataWithoutAfter4 = []
for row in trainingSet:
    if beforeTime(row[1],"4:00:00 PM"):
        dataWithoutAfter4.append(row)
trainingSet = np.array(dataWithoutAfter4)

dataWithoutAfter4 = []
for row in validationSet:
    if beforeTime(row[1],"4:00:00 PM"):
        dataWithoutAfter4.append(row)
validationSet = np.array(dataWithoutAfter4)

### Get dateTimeCandleSP with rows of (date,time,color,StrikePrice,StockPrice)

In [12]:
#Part1
def getDateTimeCandleSP(dataset):
    dateTimeCandleSP = [] #(date,time,color,StrikePrice,StockPrice)
    date = dataset[0][0]
    openPrice = dataset[0][11]
    skipDate = False
    #indices: 0=Date, 1=TimeBarStart, 13= UnderCloseAskPrice, 11=UnderOpenAskPrice
    for index,row in enumerate(dataset):
        #we are looking for the UnderOpenAskPrice(UOAP) at 9:30:00 AM (the first time it occurs) 
        #we are looking for the UnderOpenAskPrice (UOAP) at 9:36:00 AM (the last time it occurs)
        #then we subtract the 9:35:00 AM time by the 9:30:00 AM time

        if date == row[0]:
            if skipDate:
                continue
            #lastPrice = row["UnderCloseAskPrice"]
            prevRow = row
            #if time is past 9:35AM, then use the latest 9:35AM (if it doesn't exist, then use the current row's time)
            time = row[1]

            stockPrice = dateToCandle[date][3]
            candleType = dateToCandle[date][2]
            if time == "9:36:00 AM": #grab the time row for time=9:36AM to later grab the OpenTradePrice (OTP) at that time
                if candleType == "GREEN":
                    dateTimeCandleSP.append((date,time,candleType,math.ceil(stockPrice),stockPrice))
                else:
                    dateTimeCandleSP.append((date,time,candleType,math.floor(stockPrice),stockPrice))
                skipDate = True
        else:
            #we found new date, get the green/red bar for the previous date, reinitialize some variables
            date = row[0]
            openPrice = row[11]
            skipDate = False
    return dateTimeCandleSP

In [13]:
#(date,time,color,StrikePrice,StockPrice)
dateTimeCandleSP = getDateTimeCandleSP(trainingSet)
dateTimeCandleSP

[(20200103, '9:36:00 AM', 'GREEN', 323, 322.625),
 (20200106, '9:36:00 AM', 'GREEN', 322, 321.12),
 (20200108, '9:36:00 AM', 'GREEN', 324, 323.28),
 (20200110, '9:36:00 AM', 'RED', 326, 326.91),
 (20200113, '9:36:00 AM', 'RED', 326, 326.14),
 (20200115, '9:36:00 AM', 'GREEN', 328, 327.8),
 (20200117, '9:36:00 AM', 'RED', 331, 331.39),
 (20200121, '9:36:00 AM', 'GREEN', 332, 331.42),
 (20200122, '9:36:00 AM', 'GREEN', 333, 332.36),
 (20200124, '9:36:00 AM', 'RED', 332, 332.065),
 (20200127, '9:36:00 AM', 'GREEN', 324, 323.77),
 (20200129, '9:36:00 AM', 'RED', 328, 328.28),
 (20200131, '9:36:00 AM', 'RED', 326, 326.505),
 (20200203, '9:36:00 AM', 'GREEN', 324, 323.8401),
 (20200205, '9:36:00 AM', 'RED', 331, 331.98),
 (20200207, '9:36:00 AM', 'RED', 332, 332.08),
 (20200210, '9:36:00 AM', 'GREEN', 333, 332.05),
 (20200212, '9:36:00 AM', 'GREEN', 338, 337.17),
 (20200214, '9:36:00 AM', 'GREEN', 338, 337.59),
 (20200218, '9:36:00 AM', 'GREEN', 337, 336.9272),
 (20200219, '9:36:00 AM', 'GRE

In [14]:
#for debugging
#untrimmedTrainingSet = trainingSet[:]
#untrimmedTrainingSet[0]

In [15]:
#---------------comeback1
#Make the df smaller (i.e. for each date, delete rows that aren't the same CallPut and StrikePrice)
#Also delete the times that are >= 4:00:00 PM



In [16]:
#Part2
#if the candle is green then we want to buy a call
#if call then you look for the UnderOpenAskPrice at 9:36:00 AM and round that up to next whole number, this gets the Call_SP
#then we look for the Call_SP and see what the OpenTradePrice is at 9:36:00 AM

#if the candle is red then we want to buy a put
#if put then you look for the UnderOpenAskPrice at 9:36:00 AM and round that down to next whole number, this gets the Put_SP
#then we look for the Put_SP and see what the OpenTradePrice is at 9:36:00 AM

def getDateWithOTP(dataset,dateTimeCandleSP):
    dtcspIdx = 0
    dfStartIdx = 0
    dateWithOTP = [] #(date, time, CallPut, SP, OTP)
    for dtcsp in dateTimeCandleSP:
        #print("dateWithOTP: {}".format(dateWithOTP))
        date, time, color, strikeP = dtcsp[0],dtcsp[1],dtcsp[2], dtcsp[3]
        for index,row in enumerate(dataset[dfStartIdx:],dfStartIdx):
            if color == "GREEN":
                #This is call, get OpenTradePrice(OTP) at 9:36AM
                #Match Date, Time, Call=C, StrikePrice
                if row[0] == date and row[1] == time and row[3] == "C" and row[4] == strikeP:
                    dateWithOTP.append((date,time,"C",strikeP,row[6]))
                    #update for next iteration of dtcsp
                    dfStartIdx = index + 1
                    break
            else:
                #This is put, get OpenTradePrice(OTP) at 9:36AM
                #Match Date, Time, Call=P, StrikePrice
                if row[0] == date and row[1] == time and row[3] == "P" and row[4] == strikeP:
                    dateWithOTP.append((date,time,"P",strikeP,row[6]))
                    #update for next iteration of dtcsp
                    dfStartIdx = index + 1
                    break
    return dateWithOTP

In [17]:
#(date, time, CallPut, SP, OTP)
print("(date, time,CallPut, SP, OTP)")
dateWithOTP = getDateWithOTP(trainingSet,dateTimeCandleSP)
dateWithOTP

(date, time,CallPut, SP, OTP)


[(20200103, '9:36:00 AM', 'C', 323, 0.43),
 (20200106, '9:36:00 AM', 'C', 322, 0.31),
 (20200108, '9:36:00 AM', 'C', 324, 0.39),
 (20200110, '9:36:00 AM', 'P', 326, 0.21),
 (20200113, '9:36:00 AM', 'P', 326, 0.44),
 (20200115, '9:36:00 AM', 'C', 328, 0.58),
 (20200117, '9:36:00 AM', 'P', 331, 0.3),
 (20200121, '9:36:00 AM', 'C', 332, 0.23),
 (20200122, '9:36:00 AM', 'C', 333, 0.23),
 (20200124, '9:36:00 AM', 'P', 332, 0.36),
 (20200127, '9:36:00 AM', 'C', 324, 1.06),
 (20200129, '9:36:00 AM', 'P', 328, 0.69),
 (20200131, '9:36:00 AM', 'P', 326, 0.87),
 (20200203, '9:36:00 AM', 'C', 324, 1.0),
 (20200205, '9:36:00 AM', 'P', 331, 0.43),
 (20200207, '9:36:00 AM', 'P', 332, 0.71),
 (20200210, '9:36:00 AM', 'C', 333, 0.31),
 (20200212, '9:36:00 AM', 'C', 338, 0.14),
 (20200214, '9:36:00 AM', 'C', 338, 0.37),
 (20200218, '9:36:00 AM', 'C', 337, 0.65),
 (20200219, '9:36:00 AM', 'C', 338, 0.45),
 (20200221, '9:36:00 AM', 'P', 335, 0.98),
 (20200224, '9:36:00 AM', 'C', 325, 1.28),
 (20200226, '

#### For each date,SP,CallPut in dateWithOTP, find the maximum of the hightradeprice and time (b/w 9:37:00 AM to 3:59:00 PM

In [18]:
#Part4
def getDateTimeMaxHTP(dataset,dateWithOTP):
    dfStartIdx = 0
    #Date	TimeBarStart	Ticker	CallPut	Strike	ExpirationDate	OpenTradePrice	HighTradePrice	LowTradePrice	CloseTradePrice	
    dateTimeMaxHTP = [(0,0,0,0,0) for _ in range(len(dateWithOTP))] #[date,time,CallPut,SP,maxHTP]
    prevDate = dateWithOTP[0][0]
    N = len(dataset)
    curDfLen = N
    for dOTPidx, (date,_,CallPut,sp,_) in enumerate(dateWithOTP):
        #print("Checking for date={},CallPut={},sp={}".format(date,CallPut,sp))
        maxHTP = -1
        doSameDate = False
        for index,row in enumerate(dataset[dfStartIdx:],dfStartIdx):
            #print("date={}: index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(date,index,dfStartIdx,N-dfStartIdx-1))
            #print("htp={},maxHTP={},time={}".format(row[7],maxHTP,row[1]))
            if index == N-1: #index == N - dfStartIdx - 2:
                #needed this one here to check for end index, which can contain time inside the timeframe we want
                #print("if4.1-----------------------------------------")
                #case 1 - we're done with prev date
                dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                #initialize variables for next iteration of dateWithOTP
                dfStartIdx = index + 1
                prevDate = row[0]
                maxHTP = -1
                break

            if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"): 
                if index == N-1: #index == N - dfStartIdx - 2:
                    #need this one here again because the last index may have time that is outside of timeframe
                    #print("if4.1-----------------------------------------")
                    #case 1 - we're done with prev date
                    dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                    #initialize variables for next iteration of dateWithOTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    maxHTP = -1
                    break
                continue
            elif date == row[0] and CallPut == row[3] and sp == row[4]:
                #print("if1")
                #print("row: {}".format(list(row)))
                #Check if there's a new max HTP
                #print("htp={},maxHTP={},time={}".format(row[7],maxHTP,row[1]))
                if row[7] > maxHTP:
                    #print("if2")
                    #print("update for maxHTP: {}".format(row[7]))
                    maxHTP = row[7]
                    maxHTPTime = row[1]
            elif date != row[0]:
                #print("if3")
                #date doesn't match, so we're done searching for previous date OR this date has no value in df
                if maxHTP != -1: #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMaxHTP  
                    #print("if4")
                    #case 1 - we're done with prev date
                    dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                    #initialize variables for next iteration of dateWithOTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    maxHTP = -1
                    break
                elif index == N-1: #index == N - dfStartIdx - 2:
                    #print("if4.1-----------------------------------------")
                    #case 1 - we're done with prev date
                    dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                    #initialize variables for next iteration of dateWithOTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    maxHTP = -1
                    break
                else:
                    #print('if5')
                    #Case 2 - date has no value in df, so doesn't exist in dateWithOTP
                    #keep iterating until we past this date and until we found the date that matches dateWithOTP
                    dfStartIdx = index + 1
                    #print("dfStartIdx: {}".format(dfStartIdx))
                    for indexb,rowb in enumerate(dataset[dfStartIdx:],dfStartIdx):
                        if rowb[0] == date:
                            #print("if6")
                            prevDate = rowb[0]
                            dfStartIdx = indexb
                            doSameDate = True #found matching date so now iterate from that as startIdx to cover it instead of using current for loop
                            break
                    if doSameDate:
                        #print("if7")
                        break


        if doSameDate:
            #print("if7.2")
            for index,row in enumerate(dataset[dfStartIdx:],dfStartIdx):
                #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
                if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                        #print("if11")
                        #reached the last index in df, so update for maxHTP
                        #case 1 - we're done with prev date
                        dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                        #initialize variables for next iteration of dateWithOTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        maxHTP = -1
                        break

                if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                    if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                        #print("if11")
                        #reached the last index in df, so update for maxHTP
                        #case 1 - we're done with prev date
                        dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                        #initialize variables for next iteration of dateWithOTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        maxHTP = -1
                        break
                    continue
                elif date == row[0] and CallPut == row[3] and sp == row[4]:
                    #print("if8")
                    #print("row: {}".format(list(row)))
                    #Check if there's a new max HTP
                    #print("htp={},maxHTP={},time={}".format(row[7],maxHTP,row[1]))
                    if row[7] > maxHTP:
                        #print("if9")
                        #print("update for maxHTP: {}".format(row[7]))
                        maxHTP = row[7]
                        maxHTPTime = row[1]
                elif date != row[0]:
                    #date doesn't match, so we're done searching for previous date OR this date has no value in df
                    #print("if10")
                    if maxHTP != -1: #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMaxHTP 
                        #case 1 - we're done with prev date
                        dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                        #initialize variables for next iteration of dateWithOTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        maxHTP = -1
                        break
    return dateTimeMaxHTP

In [19]:
dateTimeMaxHTP = getDateTimeMaxHTP(trainingSet,dateWithOTP)
dateTimeMaxHTP

[(20200103, '12:06:00 PM', 'C', 323, 0.78),
 (20200106, '3:59:00 PM', 'C', 322, 1.65),
 (20200108, '3:29:00 PM', 'C', 324, 1.78),
 (20200110, '3:42:00 PM', 'P', 326, 0.81),
 (20200113, '9:42:00 AM', 'P', 326, 0.55),
 (20200115, '12:11:00 PM', 'C', 328, 1.0),
 (20200117, '9:40:00 AM', 'P', 331, 0.41),
 (20200121, '12:37:00 PM', 'C', 332, 0.39),
 (20200122, '9:48:00 AM', 'C', 333, 0.32),
 (20200124, '2:24:00 PM', 'P', 332, 4.6),
 (20200127, '9:52:00 AM', 'C', 324, 1.49),
 (20200129, '3:59:00 PM', 'P', 328, 1.43),
 (20200131, '3:42:00 PM', 'P', 326, 5.15),
 (20200203, '9:54:00 AM', 'C', 324, 1.64),
 (20200205, '9:41:00 AM', 'P', 331, 0.69),
 (20200207, '2:13:00 PM', 'P', 332, 0.78),
 (20200210, '3:59:00 PM', 'C', 333, 1.64),
 (20200212, '9:40:00 AM', 'C', 338, 0.2),
 (20200214, '9:40:00 AM', 'C', 338, 0.32),
 (20200218, '9:42:00 AM', 'C', 337, 0.65),
 (20200219, '2:45:00 PM', 'C', 338, 1.1),
 (20200221, '1:55:00 PM', 'P', 335, 2.41),
 (20200224, '9:49:00 AM', 'C', 325, 1.78),
 (20200226, 

#### Find the minimum of the option (LowTradePrice) from 9:37:00 AM to the maxHTP time
#### If the max time occurs at 9:37:00 AM then find the minimum LowTradePrice of the option from 9:37:00 AM to the end of the day

In [20]:
#Part3
def getDateTimeMinLTP(dataset,dateTimeMaxHTP):
    #Learned to print with end="\r",flush=True - works only for 1 changing variable per print.
    dfStartIdx = 0
    #Date	TimeBarStart	Ticker	CallPut	Strike	ExpirationDate	OpenTradePrice	HighTradePrice	LowTradePrice	CloseTradePrice	
    dateTimeMinLTP = [(0,0,0,0,0) for _ in range(len(dateTimeMaxHTP))] #[date,time,CallPut,SP,minLTP]
    prevDate = dateTimeMaxHTP[0][0]
    N = len(dataset)
    curDfLen = N
    printCount = 0
    for dTMidx, (date,maxHTPTime,CallPut,sp,_) in enumerate(dateTimeMaxHTP):
        if dfStartIdx >= N-1: #previously there was an error bc when we started using powershell data, our dates can go beyond what dates we have in df, so we get stuck
            break
        while int(prevDate) < date:
            #if printCount % 10000 == 0 or printCount >= 520000:
            #    print("1. dfStartIdx: {}".format(dfStartIdx),end="\r",flush=True)
            #printCount += 1
            #if the row's date is not part of our dateTimeMaxHTP, then we keep moving to next row
            dfStartIdx += 1
            if dfStartIdx >= N - 1:
                break
            #print("dfStartIdx={}".format(dfStartIdx))
            #print("N = {}".format(N))
            #print("dfStartIdx={},dataset[dfStartIdx]={}".format(dfStartIdx,dataset[dfStartIdx]))
            prevDate = dataset[dfStartIdx][0]


        #print("Checking for date={},maxHTPTime={},CallPut={},sp={}".format(date,maxHTPTime,CallPut,sp))
        minLTP = float("inf")
        doSameDate = False

        if maxHTPTime == "9:37:00 AM":
            #print("inside maxHTPTime == 9:37:00 AM")
            #if the max time occurs at 9:37:00 AM then find the minimum LowTradePrice of the option from 9:37:00 AM to the end of the day
            for index,row in enumerate(dataset[dfStartIdx:],dfStartIdx):
                #print("2. index={},row={}: ".format(index,row), end="\r", flush=True)
                #if printCount % 10000 == 0 or printCount >= 520000:
                #    print("2. index={}: ".format(index), end="\r", flush=True)
                #printCount += 1
                #print("index,dfStartIdx,N-dfStartIdx-1,time: {},{},{},{}".format(index,dfStartIdx,N-dfStartIdx-1,row[1]))
                #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                if index == N-1: #index == N - dfStartIdx - 2:
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break

                #want 9:37AM to end of day
                #otherwise toss out this row
                if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                    if index == N-1: #index == N - dfStartIdx - 2:
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break
                    continue
                elif date == row[0] and CallPut == row[3] and sp == row[4]:
                    #print("row: {}".format(list(row)))
                    #Check if there's a new max HTP
                    #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                    if row[8] < minLTP:
                        #print("update for minLTP: {}".format(row[8]))
                        minLTP = row[8]
                        minLTPTime = row[1]
                elif date != row[0]:
                    #date doesn't match, so we're done searching for previous date OR this date has no value in df
                    if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break
                    elif index == N-1: #index == N - dfStartIdx - 2:
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break
                    else:
                        #print('if5 - date has no value in df')
                        #Case 2 - date has no value in df, so doesn't exist in dateTimeMaxHTP
                        #keep iterating until we past this date and until we found the date that matches dateTimeMaxHTP
                        dfStartIdx = index + 1
                        #print("dfStartIdx: {}".format(dfStartIdx))
                        for indexb,rowb in enumerate(dataset[dfStartIdx:],dfStartIdx):
                            #if printCount % 10000 == 0 or printCount >= 520000:
                            #    print("3. indexb={},rowb={}".format(indexb,rowb),end="\r", flush=True)
                            #printCount += 1
                            if rowb[0] == date:
                                prevDate = rowb[0]
                                dfStartIdx = indexb
                                doSameDate = True #found matching date so now iterate from that as startIdx to cover it instead of using current for loop
                                break
                        if doSameDate:
                            break


                if doSameDate:
                    for index,row in enumerate(dataset[dfStartIdx:],dfStartIdx):
                        #if printCount % 10000 == 0 or printCount >= 520000:
                        #    print("4. index={}".format(index))
                        #printCount += 1
                        #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
                        if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                            #case 1 - we're done with prev date
                            #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                            dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                            #initialize variables for next iteration of dateTimeMaxHTP
                            dfStartIdx = index + 1
                            prevDate = row[0]
                            minLTP = float("inf")
                            break

                        #want 9:37AM to end of day
                        #otherwise toss out this row
                        if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                            if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                                #reached the last index in df, so update for minLTP
                                #case 1 - we're done with prev date
                                #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                                dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                                #initialize variables for next iteration of dateTimeMaxHTP
                                dfStartIdx = index + 1
                                prevDate = row[0]
                                minLTP = float("inf")
                                break
                            continue
                        elif date == row[0] and CallPut == row[3] and sp == row[4]:
                            #print("row: {}".format(list(row)))
                            #Check if there's a new max HTP
                            #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                            if row[8] < minLTP:
                                #print("update for minLTP: {}".format(row[8]))
                                minLTP = row[8]
                                minLTPTime = row[1]
                        elif date != row[0]:
                            #date doesn't match, so we're done searching for previous date OR this date has no value in df
                            if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP 
                                #case 1 - we're done with prev date
                                #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                                dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                                #initialize variables for next iteration of dateTimeMaxHTP
                                dfStartIdx = index + 1
                                prevDate = row[0]
                                minLTP = float("inf")
                                break
        else:
            #print("NOT inside maxHTPTime == 9:37:00 AM")
            #max occurs after 9:37AM - find the minimum of the option (LowTradePrice) from 9:37:00 AM to the max HTP time
            for index,row in enumerate(dataset[dfStartIdx:],dfStartIdx):
                #if printCount % 10000 == 0 or printCount >= 520000:
                #    print("5. index={}".format(index))
                #printCount += 1
                #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
                #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                if index == N-1: #index == N - dfStartIdx - 2:
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break

                #want 9:37AM to maxHTP time
                #toss out others
                if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],maxHTPTime):
                    if index == N-1: #index == N - dfStartIdx - 2:
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break
                    continue
                elif date == row[0] and CallPut == row[3] and sp == row[4]:
                    #print("row: {}".format(list(row)))
                    #Check if there's a new max HTP
                    #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                    if row[8] < minLTP:
                        #print("update for minLTP: {}".format(row[8]))
                        minLTP = row[8]
                        minLTPTime = row[1]
                elif date != row[0]:
                    #date doesn't match, so we're done searching for previous date OR this date has no value in df
                    if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break
                    elif index == N-1: #index == N - dfStartIdx - 2:
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break
                    else:
                        #print('if5')
                        #Case 2 - date has no value in df, so doesn't exist in dateTimeMaxHTP
                        #keep iterating until we past this date and until we found the date that matches dateTimeMaxHTP
                        dfStartIdx = index + 1
                        #print("dfStartIdx: {}".format(dfStartIdx))
                        for indexb,rowb in enumerate(dataset[dfStartIdx:],dfStartIdx):
                            if rowb[0] == date:
                                prevDate = rowb[0]
                                dfStartIdx = indexb
                                doSameDate = True #found matching date so now iterate from that as startIdx to cover it instead of using current for loop
                                break
                        if doSameDate:
                            break


                if doSameDate:
                    for index,row in enumerate(dataset[dfStartIdx:],dfStartIdx):
                        #if printCount % 10000 == 0 or printCount >= 520000:
                        #    print("6. index={}".format(index))
                        #printCount += 1
                        #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
                        if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                            #case 1 - we're done with prev date
                            #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                            #print("prevDate={},minLTPTime={},CallPut={},sp={},minLTP={}".format(prevDate,minLTPTime,CallPut,sp,minLTP))
                            dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                            #initialize variables for next iteration of dateTimeMaxHTP
                            dfStartIdx = index + 1
                            prevDate = row[0]
                            minLTP = float("inf")
                            break

                        if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],maxHTPTime):
                            if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                                #reached the last index in df, so update for minLTP
                                #case 1 - we're done with prev date
                                dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                                #initialize variables for next iteration of dateTimeMaxHTP
                                dfStartIdx = index + 1
                                prevDate = row[0]
                                minLTP = float("inf")
                                break
                            continue
                        elif date == row[0] and CallPut == row[3] and sp == row[4]:
                            #print("row: {}".format(list(row)))
                            #Check if there's a new max HTP
                            #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                            if row[8] < minLTP:
                                #print("update for minLTP: {}".format(row[8]))
                                minLTP = row[8]
                                minLTPTime = row[1]
                        elif date != row[0]:
                            #date doesn't match, so we're done searching for previous date OR this date has no value in df
                            if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP 
                                #case 1 - we're done with prev date
                                dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                                #initialize variables for next iteration of dateTimeMaxHTP
                                dfStartIdx = index + 1
                                prevDate = row[0]
                                minLTP = float("inf")
                                break
    return dateTimeMinLTP

In [21]:
dateTimeMinLTP = getDateTimeMinLTP(trainingSet,dateTimeMaxHTP)
dateTimeMinLTP

[(20200103, '9:38:00 AM', 'C', 323, 0.35),
 (20200106, '9:38:00 AM', 'C', 322, 0.24),
 (20200108, '9:39:00 AM', 'C', 324, 0.31),
 (20200110, '1:44:00 PM', 'P', 326, 0.07),
 (20200113, '9:42:00 AM', 'P', 326, 0.34),
 (20200115, '9:37:00 AM', 'C', 328, 0.55),
 (20200117, '9:38:00 AM', 'P', 331, 0.34),
 (20200121, '9:43:00 AM', 'C', 332, 0.12),
 (20200122, '9:37:00 AM', 'C', 333, 0.22),
 (20200124, '9:59:00 AM', 'P', 332, 0.34),
 (20200127, '9:37:00 AM', 'C', 324, 1.0),
 (20200129, '2:04:00 PM', 'P', 328, 0.24),
 (20200131, '9:39:00 AM', 'P', 326, 0.6),
 (20200203, '9:41:00 AM', 'C', 324, 1.02),
 (20200205, '9:38:00 AM', 'P', 331, 0.46),
 (20200207, '12:24:00 PM', 'P', 332, 0.09),
 (20200210, '9:50:00 AM', 'C', 333, 0.26),
 (20200212, '9:37:00 AM', 'C', 338, 0.14),
 (20200214, '9:37:00 AM', 'C', 338, 0.25),
 (20200218, '9:37:00 AM', 'C', 337, 0.43),
 (20200219, '9:38:00 AM', 'C', 338, 0.43),
 (20200221, '12:05:00 PM', 'P', 335, 0.65),
 (20200224, '9:42:00 AM', 'C', 325, 1.32),
 (20200226,

#### Now get mergedOtpLtpHtp 

In [22]:
dateWithOTP

[(20200103, '9:36:00 AM', 'C', 323, 0.43),
 (20200106, '9:36:00 AM', 'C', 322, 0.31),
 (20200108, '9:36:00 AM', 'C', 324, 0.39),
 (20200110, '9:36:00 AM', 'P', 326, 0.21),
 (20200113, '9:36:00 AM', 'P', 326, 0.44),
 (20200115, '9:36:00 AM', 'C', 328, 0.58),
 (20200117, '9:36:00 AM', 'P', 331, 0.3),
 (20200121, '9:36:00 AM', 'C', 332, 0.23),
 (20200122, '9:36:00 AM', 'C', 333, 0.23),
 (20200124, '9:36:00 AM', 'P', 332, 0.36),
 (20200127, '9:36:00 AM', 'C', 324, 1.06),
 (20200129, '9:36:00 AM', 'P', 328, 0.69),
 (20200131, '9:36:00 AM', 'P', 326, 0.87),
 (20200203, '9:36:00 AM', 'C', 324, 1.0),
 (20200205, '9:36:00 AM', 'P', 331, 0.43),
 (20200207, '9:36:00 AM', 'P', 332, 0.71),
 (20200210, '9:36:00 AM', 'C', 333, 0.31),
 (20200212, '9:36:00 AM', 'C', 338, 0.14),
 (20200214, '9:36:00 AM', 'C', 338, 0.37),
 (20200218, '9:36:00 AM', 'C', 337, 0.65),
 (20200219, '9:36:00 AM', 'C', 338, 0.45),
 (20200221, '9:36:00 AM', 'P', 335, 0.98),
 (20200224, '9:36:00 AM', 'C', 325, 1.28),
 (20200226, '

In [23]:
#[date,time,CallPut,sp,minLTP]
dateTimeMinLTP

[(20200103, '9:38:00 AM', 'C', 323, 0.35),
 (20200106, '9:38:00 AM', 'C', 322, 0.24),
 (20200108, '9:39:00 AM', 'C', 324, 0.31),
 (20200110, '1:44:00 PM', 'P', 326, 0.07),
 (20200113, '9:42:00 AM', 'P', 326, 0.34),
 (20200115, '9:37:00 AM', 'C', 328, 0.55),
 (20200117, '9:38:00 AM', 'P', 331, 0.34),
 (20200121, '9:43:00 AM', 'C', 332, 0.12),
 (20200122, '9:37:00 AM', 'C', 333, 0.22),
 (20200124, '9:59:00 AM', 'P', 332, 0.34),
 (20200127, '9:37:00 AM', 'C', 324, 1.0),
 (20200129, '2:04:00 PM', 'P', 328, 0.24),
 (20200131, '9:39:00 AM', 'P', 326, 0.6),
 (20200203, '9:41:00 AM', 'C', 324, 1.02),
 (20200205, '9:38:00 AM', 'P', 331, 0.46),
 (20200207, '12:24:00 PM', 'P', 332, 0.09),
 (20200210, '9:50:00 AM', 'C', 333, 0.26),
 (20200212, '9:37:00 AM', 'C', 338, 0.14),
 (20200214, '9:37:00 AM', 'C', 338, 0.25),
 (20200218, '9:37:00 AM', 'C', 337, 0.43),
 (20200219, '9:38:00 AM', 'C', 338, 0.43),
 (20200221, '12:05:00 PM', 'P', 335, 0.65),
 (20200224, '9:42:00 AM', 'C', 325, 1.32),
 (20200226,

In [24]:
dateTimeMaxHTP

[(20200103, '12:06:00 PM', 'C', 323, 0.78),
 (20200106, '3:59:00 PM', 'C', 322, 1.65),
 (20200108, '3:29:00 PM', 'C', 324, 1.78),
 (20200110, '3:42:00 PM', 'P', 326, 0.81),
 (20200113, '9:42:00 AM', 'P', 326, 0.55),
 (20200115, '12:11:00 PM', 'C', 328, 1.0),
 (20200117, '9:40:00 AM', 'P', 331, 0.41),
 (20200121, '12:37:00 PM', 'C', 332, 0.39),
 (20200122, '9:48:00 AM', 'C', 333, 0.32),
 (20200124, '2:24:00 PM', 'P', 332, 4.6),
 (20200127, '9:52:00 AM', 'C', 324, 1.49),
 (20200129, '3:59:00 PM', 'P', 328, 1.43),
 (20200131, '3:42:00 PM', 'P', 326, 5.15),
 (20200203, '9:54:00 AM', 'C', 324, 1.64),
 (20200205, '9:41:00 AM', 'P', 331, 0.69),
 (20200207, '2:13:00 PM', 'P', 332, 0.78),
 (20200210, '3:59:00 PM', 'C', 333, 1.64),
 (20200212, '9:40:00 AM', 'C', 338, 0.2),
 (20200214, '9:40:00 AM', 'C', 338, 0.32),
 (20200218, '9:42:00 AM', 'C', 337, 0.65),
 (20200219, '2:45:00 PM', 'C', 338, 1.1),
 (20200221, '1:55:00 PM', 'P', 335, 2.41),
 (20200224, '9:49:00 AM', 'C', 325, 1.78),
 (20200226, 

In [25]:
numValidDates = len(dateWithOTP)
mergedOtpLtpHtp = [(0,"Z",0,("Z",0),("Z",0),("Z",0))] * numValidDates #(date,CallPut,SP,(time1,OTP),(time2,minLTP),(time3,maxHTP))
for index in range(numValidDates):
    date = dateWithOTP[index][0]
    CallPut = dateWithOTP[index][2]
    SP = dateWithOTP[index][3]
    time1, OTP = dateWithOTP[index][1], dateWithOTP[index][4]
    time2, minLTP = dateTimeMinLTP[index][1], dateTimeMinLTP[index][4]
    time3, maxHTP = dateTimeMaxHTP[index][1], dateTimeMaxHTP[index][4]
    mergedOtpLtpHtp[index] = (date,CallPut,SP,(time1,OTP),(time2,minLTP),(time3,maxHTP))

In [26]:
mergedOtpLtpHtp

[(20200103,
  'C',
  323,
  ('9:36:00 AM', 0.43),
  ('9:38:00 AM', 0.35),
  ('12:06:00 PM', 0.78)),
 (20200106,
  'C',
  322,
  ('9:36:00 AM', 0.31),
  ('9:38:00 AM', 0.24),
  ('3:59:00 PM', 1.65)),
 (20200108,
  'C',
  324,
  ('9:36:00 AM', 0.39),
  ('9:39:00 AM', 0.31),
  ('3:29:00 PM', 1.78)),
 (20200110,
  'P',
  326,
  ('9:36:00 AM', 0.21),
  ('1:44:00 PM', 0.07),
  ('3:42:00 PM', 0.81)),
 (20200113,
  'P',
  326,
  ('9:36:00 AM', 0.44),
  ('9:42:00 AM', 0.34),
  ('9:42:00 AM', 0.55)),
 (20200115,
  'C',
  328,
  ('9:36:00 AM', 0.58),
  ('9:37:00 AM', 0.55),
  ('12:11:00 PM', 1.0)),
 (20200117,
  'P',
  331,
  ('9:36:00 AM', 0.3),
  ('9:38:00 AM', 0.34),
  ('9:40:00 AM', 0.41)),
 (20200121,
  'C',
  332,
  ('9:36:00 AM', 0.23),
  ('9:43:00 AM', 0.12),
  ('12:37:00 PM', 0.39)),
 (20200122,
  'C',
  333,
  ('9:36:00 AM', 0.23),
  ('9:37:00 AM', 0.22),
  ('9:48:00 AM', 0.32)),
 (20200124,
  'P',
  332,
  ('9:36:00 AM', 0.36),
  ('9:59:00 AM', 0.34),
  ('2:24:00 PM', 4.6)),
 (20200127

In [27]:
value = 1 - ((2.26 - 2.22) / 2.26 * 100)
formatted_string = "{:.3f}".format(value)
print(formatted_string + "%")

-0.770%


In [28]:
import collections
dropFromOTP = collections.defaultdict(int) #part 1
buyToMax = collections.defaultdict(int) #part 2
numValidDates = len(mergedOtpLtpHtp)
margin = 5 #this is our margin for our %
for index in range(numValidDates):
    time1, otp = mergedOtpLtpHtp[index][3][0], mergedOtpLtpHtp[index][3][1]
    time2, minLTP = mergedOtpLtpHtp[index][4][0], mergedOtpLtpHtp[index][4][1]
    time3, maxHTP = mergedOtpLtpHtp[index][5][0], mergedOtpLtpHtp[index][5][1]
    if afterTime(time2,time3):
        #failure date, we lose 50%
        dropPerc = -50
    else:
        #dropPerc = math.ceil((otp - minLTP) / otp * 100)
        dropPerc = (otp - minLTP) / otp * 100 # 93.222 --> 95
        dropPerc =  margin * math.ceil(dropPerc/margin)
    #print(otp,minLTP,dropPerc)
    if dropPerc > 0:
        dropFromOTP[dropPerc] += 1
    print("otp={}, minLTP={}, dropPerc={}".format(otp,minLTP,dropPerc))
    
mostOccurredDropPerc = max(dropFromOTP, key=dropFromOTP.get)
print("mostOccurredDropPerc:{}".format(mostOccurredDropPerc))

for index in range(numValidDates):
    time1, otp = mergedOtpLtpHtp[index][3][0], mergedOtpLtpHtp[index][3][1]
    time2, minLTP = mergedOtpLtpHtp[index][4][0], mergedOtpLtpHtp[index][4][1]
    time3, maxHTP = mergedOtpLtpHtp[index][5][0], mergedOtpLtpHtp[index][5][1]
    #print("otp={}, minLTP={}, mostOccurredDropPerc={}, buyPriceGeneral={}".format(otp,minLTP,mostOccurredDropPerc,buyPriceGeneral))
    
    #percent profit from buyPriceGeneral --> maxHTP
    if afterTime(time2,time3):
        buyPriceGeneral = 0
        profitPerc = -50
    else:
        buyPriceGeneral = otp - mostOccurredDropPerc/100*otp
        profitPerc = math.floor((maxHTP - buyPriceGeneral) / maxHTP * 100)
    if buyPriceGeneral*0.5 >= minLTP:
        #if our minLTP can still drop very low (past 50% our buy) then we have a failure
        profitPerc = -50
    #print("otp={}, minLTP={}, mostOccurredDropPerc={}, buyPriceGeneral={}, profitPerc={}".format(otp,minLTP,mostOccurredDropPerc,buyPriceGeneral,profitPerc))
    buyToMax[profitPerc] += 1
temp = buyToMax[-50]
del buyToMax[-50]
mostOccuredBuyProfit = max(buyToMax, key=buyToMax.get) #excludes -50
print("mostOccuredBuyProfit:{}".format(mostOccuredBuyProfit))
buyToMax[-50] = temp

otp=0.43, minLTP=0.35, dropPerc=20
otp=0.31, minLTP=0.24, dropPerc=25
otp=0.39, minLTP=0.31, dropPerc=25
otp=0.21, minLTP=0.07, dropPerc=70
otp=0.44, minLTP=0.34, dropPerc=25
otp=0.58, minLTP=0.55, dropPerc=10
otp=0.3, minLTP=0.34, dropPerc=-10
otp=0.23, minLTP=0.12, dropPerc=50
otp=0.23, minLTP=0.22, dropPerc=5
otp=0.36, minLTP=0.34, dropPerc=10
otp=1.06, minLTP=1.0, dropPerc=10
otp=0.69, minLTP=0.24, dropPerc=70
otp=0.87, minLTP=0.6, dropPerc=35
otp=1.0, minLTP=1.02, dropPerc=0
otp=0.43, minLTP=0.46, dropPerc=-5
otp=0.71, minLTP=0.09, dropPerc=90
otp=0.31, minLTP=0.26, dropPerc=20
otp=0.14, minLTP=0.14, dropPerc=0
otp=0.37, minLTP=0.25, dropPerc=35
otp=0.65, minLTP=0.43, dropPerc=35
otp=0.45, minLTP=0.43, dropPerc=5
otp=0.98, minLTP=0.65, dropPerc=35
otp=1.28, minLTP=1.32, dropPerc=0
otp=1.73, minLTP=1.92, dropPerc=-10
otp=3.87, minLTP=3.7, dropPerc=5
otp=3.17, minLTP=3.1, dropPerc=5
otp=1.58, minLTP=0.85, dropPerc=50
otp=2.87, minLTP=0.46, dropPerc=85
otp=3.24, minLTP=0.01, dropPerc

In [29]:
print("dropFromOTP: {}".format(dropFromOTP))
print("buyToMax: {}".format(buyToMax))
print("mostOccurredDropPerc: {}%".format(mostOccurredDropPerc))
print("mostOccuredBuyProfit: {}%".format(mostOccuredBuyProfit))

dropFromOTP: defaultdict(<class 'int'>, {20: 6, 25: 5, 70: 2, 10: 7, 50: 2, 5: 6, 35: 8, 90: 1, 85: 1, 55: 2, 45: 2, 40: 5, 60: 1, 80: 1, 15: 4, 65: 1, 30: 2})
buyToMax: defaultdict(<class 'int'>, {64: 2, 87: 2, 85: 1, 83: 2, 48: 2, 62: 1, 52: 2, 61: 4, 53: 3, 94: 1, 68: 1, 89: 1, 60: 1, 59: 1, 54: 2, 24: 1, 35: 1, 73: 4, 65: 2, 42: 1, 40: 2, 75: 1, 67: 1, 69: 1, 51: 1, 55: 1, 32: 1, 50: 1, 38: 2, 26: 1, 43: 2, 25: 1, 82: 1, 19: 1, 77: 2, 34: 1, 44: 1, 76: 1, 72: 1, 80: 1, 84: 1, 58: 1, 88: 1, 27: 1, -50: 7})
mostOccurredDropPerc: 35%
mostOccuredBuyProfit: 61%


In [30]:
#UPDATED1
#takes in a dataset, and outputs date, color, SP, OTP, minLTP, and maxHTP
def datasetToMerged(dataset):
    #PART 1 GETS Color and StrikePrice
    dateTimeCandleSP = getDateTimeCandleSP(dataset)
    #PART 2 GETS OTP
    dateWithOTP = getDateWithOTP(dataset,dateTimeCandleSP)
    #PART 4 GETS maxHTP
    dateTimeMaxHTP = getDateTimeMaxHTP(dataset,dateWithOTP)
    #PART 3 GETS minLTP
    dateTimeMinLTP = getDateTimeMinLTP(dataset,dateTimeMaxHTP)

    #MERGE ALL 4 PARTS
    #part 1 - dateTimeCandleSP = [] #(date,time,color,StrikePrice,StockPrice)
    #part 2 - dateWithOTP = [] #(date, time, CallPut, SP, OTP)
    #part 3 - dateTimeMinLTP = [(0,0,0,0,0) for _ in range(len(dateTimeMaxHTP))] #[date,time,CallPut,SP,minLTP]
    #part 4 - dateTimeMaxHTP = [(0,0,0,0,0) for _ in range(len(dateWithOTP))] #[date,time,CallPut,SP,maxHTP]
    numDates = len(dateTimeCandleSP)
    result = [("Z","Z",-1,-1,-1,-1) for _ in range(numDates)] #(date,color,SP,OTP,minLTP,maxHTP)
    for index in range(numDates):
        #print("dateTimeCandleSP[index]: {}".format(dateTimeCandleSP[index]))
        #print("dateWithOTP[index]: {}".format(dateWithOTP[index]))
        #print("next")
        date = dateTimeCandleSP[index][0]
        color = dateTimeCandleSP[index][2]
        SP = dateWithOTP[index][3]
        OTP = dateWithOTP[index][4]
        minLTP = dateTimeMinLTP[index][4]
        maxHTP = dateTimeMaxHTP[index][4]
        result[index] = (date,color,SP,OTP,minLTP,maxHTP)
    return result

In [31]:
#dfValidation = pd.read_csv('Jan 2020 - Dec 2020 merged SPY.csv')
result = datasetToMerged(validationSet)
result #(date,color,SP,OTP,minLTP,maxHTP)
#error1

[(20200617, 'RED', 313, 1.47, 1.26, 2.25),
 (20200619, 'RED', 313, 0.81, 0.78, 5.8),
 (20200622, 'RED', 307, 1.35, 1.35, 1.67),
 (20200624, 'RED', 309, 0.84, 0.7, 6.13),
 (20200626, 'RED', 305, 1.08, 0.86, 5.72),
 (20200629, 'RED', 300, 1.35, 1.36, 2.14),
 (20200630, 'GREEN', 305, 1.08, 0.87, 5.15),
 (20200701, 'GREEN', 311, 1.13, 0.79, 1.2),
 (20200702, 'GREEN', 316, 0.78, 0.56, 0.82),
 (20200706, 'GREEN', 317, 0.7, 0.52, 0.9),
 (20200708, 'GREEN', 316, 0.62, 0.64, 1.13),
 (20200710, 'RED', 313, 1.16, 0.99, 1.09),
 (20200713, 'GREEN', 321, 0.52, 0.32, 1.85),
 (20200715, 'RED', 322, 1.09, 0.93, 3.0),
 (20200717, 'RED', 321, 0.86, 0.85, 1.15),
 (20200720, 'GREEN', 322, 0.8, 0.45, 3.0),
 (20200722, 'GREEN', 326, 0.56, 0.17, 1.23),
 (20200724, 'RED', 320, 1.16, 1.35, 1.79),
 (20200727, 'RED', 321, 0.98, 0.91, 1.0),
 (20200729, 'GREEN', 323, 0.88, 0.73, 2.72),
 (20200731, 'RED', 325, 0.97, 0.95, 3.71),
 (20200803, 'RED', 328, 0.82, 0.95, 1.06),
 (20200805, 'RED', 331, 0.6, 0.53, 0.61),
 (2

In [32]:
profits = [] #(date,profit)
printCount = 0 #for debug
#these are computed previously from a training dataset
#mostOccurredDropPerc:5
#mostOccuredBuyProfit:39

netProfit = 1
dfStartIdx = 0
#indices: 0=Date, 1=TimeBarStart, 13= UnderCloseAskPrice, 11=UnderOpenAskPrice
for date,color,SP,OTP,minLTP,maxHTP in result:
    print("Finding profit for date={}".format(date))
    #keep iterating row startIdx forward until we find matching date
    for index,row in enumerate(validationSet[dfStartIdx:],dfStartIdx):
        if row[0] != date:
            continue
        else:
            dfStartIdx = index
            break
    
    if color == "GREEN":
        callType = "C"
    else:
        callType = "P"
    
    #if our calculated price is > LTP then we successfully bought the option
    buyPrice = OTP * (1-mostOccurredDropPerc/100)
    bought = False
    sold = False
    for index,row in enumerate(validationSet[dfStartIdx:],dfStartIdx):
        #printCount += 1
        #if printCount % 10**4:
        #    print("date={},time={}".format(row[0],row[1]))
        #print("date={},time={}".format(row[0],row[1]))
        
        #if we past the date that we're interested in, then break and move on to next interested date
        if date != row[0]:
            break
        
        if callType == row[3] and SP == row[4] and afterTime(row[1],"9:36:00 AM") and beforeTime(row[1],"4:00:00 PM"):
            if row[1] == "3:59:00 PM": #bug2
                print("has 3:59PM")
                #This is our last time of the date that we're considering, so if we bought something, then sell it
                if bought and not sold:
                    #sell what we bought at OTP from 3:59:00 PM
                    profit = row[6]/buyPrice #currentTimeOTP/buyPrice
                    netProfit *= profit
                    profits.append((date,profit))
                    print("Sold at: date={}, time={}, profit={}, buyPrice={}, sellPrice={}".format(date,row[1],profit,buyPrice,row[6]))
                    #buyPrice --> OTP
                    #5 --> 11 #2.2
                    #11 --> 5 #0.45
                    #[1.3,0.9,1.2] 1.3*0.9*1.2*(0.5)^failcount*(mostOccuredBuyProfit)^successCount
                    #1.3,1.5,0.9,0.5^2,1.39^5
                    #1.3*1.5
                    
                #reinitialize variables for next date
                dfStartIdx = index #+ 1
                print("1. done with date={}".format(row[0]))
                break
            
            #if this date doesn't have 3:59PM with the desired constraints, then we will pretend that we didn't buy previously
            #elif afterTime(row[1],"3:59:00 PM"):
            #    dfStartIdx = index
            #    print("2. done with date={}".format(row[0]))
            #    break
            elif bought and not sold:
                #see if this is sellable (i.e. sellGoal < HTP)
                if sellGoal < row[7]:
                    #successfully sold at this price, so obtain the profit
                    #our profit % is mostOccuredBuyProfit, so tally the success
                    #successCount += 1
                    profit = 1+mostOccuredBuyProfit/100
                    netProfit *= profit
                    profits.append((date,profit))
                    print("Sold at profit: date={}, time={}, profit={}, boughtAt={}, soldAt={}, HTP={}".format(date,row[1],profit,buyPrice,sellGoal,row[7]))
                    sold = True
                elif buyPrice * 0.5 >= row[8]:
                    #we hit -50% limit, so sell it for that loss before we lose more
                    netProfit *= 0.5
                    profit = 0.5
                    profits.append((date,profit))
                    print("Sold at loss: date={}, time={}, profit={}, buyPrice={}, sellPrice={}, LTP={}".format(date,row[1],profit,buyPrice,buyPrice/2,row[8]))
                    sold = True
                #otherwise keep holding
            
            if not bought and not sold:
                if buyPrice > row[8]: #buyprice > LTP then we successfully bought the option
                    print("buyPrice={},LTP={}".format(buyPrice,row[8]))
                    bought = True
                    print("bought at date={},time={}".format(date,row[1]))
                    #We now look to sell it for buyPrice*(1+mostOccuredBuyProfit)
                    sellGoal = buyPrice*(1+mostOccuredBuyProfit/100)
        
        #this part can't be used bc we already trimmed out >= 4:00:00 PM
        """
        #if we past the desired time, then end the day
        if afterTime(row[1],"3:59:00 PM"):
            #reinitialize variables for next date
            dfStartIdx = index
            print("1. done with date={}".format(row[0]))
            break
        """

Finding profit for date=20200617
buyPrice=0.9555,LTP=0.79
bought at date=20200617,time=12:32:00 PM
Sold at loss: date=20200617, time=1:28:00 PM, profit=0.5, buyPrice=0.9555, sellPrice=0.47775, LTP=0.47
has 3:59PM
1. done with date=20200617
Finding profit for date=20200619
has 3:59PM
1. done with date=20200619
Finding profit for date=20200622
buyPrice=0.8775000000000001,LTP=0.87
bought at date=20200622,time=9:53:00 AM
Sold at loss: date=20200622, time=12:00:00 PM, profit=0.5, buyPrice=0.8775000000000001, sellPrice=0.43875000000000003, LTP=0.28
has 3:59PM
1. done with date=20200622
Finding profit for date=20200624
has 3:59PM
1. done with date=20200624
Finding profit for date=20200626
has 3:59PM
1. done with date=20200626
Finding profit for date=20200629
buyPrice=0.8775000000000001,LTP=0.18
bought at date=20200629,time=12:00:00 PM
Sold at loss: date=20200629, time=12:01:00 PM, profit=0.5, buyPrice=0.8775000000000001, sellPrice=0.43875000000000003, LTP=0.18
has 3:59PM
1. done with date=202

In [33]:
profits

[(20200617, 0.5),
 (20200622, 0.5),
 (20200629, 0.5),
 (20200701, 0.5),
 (20200702, 0.5),
 (20200706, 0.5),
 (20200708, 0.5),
 (20200710, 0.5),
 (20200713, 1.6099999999999999),
 (20200715, 1.6099999999999999),
 (20200717, 0.5),
 (20200720, 1.6099999999999999),
 (20200722, 0.5),
 (20200724, 1.6099999999999999),
 (20200727, 0.5),
 (20200731, 0.5),
 (20200803, 0.5),
 (20200805, 0.5),
 (20200807, 1.6099999999999999),
 (20200810, 0.5),
 (20200814, 0.5),
 (20200817, 0.5),
 (20200819, 0.5),
 (20200824, 0.5),
 (20200826, 0.5),
 (20200828, 0.5),
 (20200831, 0.5),
 (20200902, 1.6099999999999999),
 (20200904, 0.5),
 (20200908, 0.5),
 (20200909, 0.5),
 (20200911, 1.6099999999999999),
 (20200914, 0.5),
 (20200916, 1.6099999999999999),
 (20200921, 1.6099999999999999),
 (20200928, 1.6099999999999999),
 (20200930, 0.5),
 (20201002, 1.6099999999999999),
 (20201007, 1.6099999999999999),
 (20201009, 1.6099999999999999),
 (20201012, 0.5),
 (20201014, 1.6099999999999999),
 (20201016, 0.5),
 (20201019, 0.5)

In [34]:
mostOccurredDropPerc
mostOccuredBuyProfit

61

In [35]:
#netProfit is the fraction of the money left (e.g. netProfit=0.42 means we have 0.42*initialMoney left (we lost money), and netprofit=1.38 means we have 1.38*initialMoney (we profited 38%))
netProfit

3.382481072989428e-11

## 